## Procesamiento de Lenguaje Natural

*Aplicación de N-gramas: **generación de frases mañaneras*** 

# ***Obtención de datos***

### **Equipo:**

- Giottonini Herrera Enrique Alejandro
- Burruel Durán Luis Andrés
- Villalba Miranda Jesús Abraham

**Fuentes**
* Los datos fueron obtenidos de [@nostradata ](https://www.nostrodata.com) en [`GitHub`](https://github.com/NOSTRODATA/conferencias_matutinas_amlo)
---

## Objetivo

El objetivo de esta libreta es obtener el `corpus` que utilizaremos para la elaboración de nuestra aplicación de **n-gramas**.

Para la obtención de los datos utilizamos la libreria de [`PyGitHub`](https://pygithub.readthedocs.io/en/latest/introduction.html), lo que nos permite interactuar con el repositorio de *@nostradata* por medio de una REST API.

Ademas, utilizaremos `Pandas`, lo que nos permitira cargar los datos mediante una *URL* en un `DataFrame`.


In [1]:
from github import Github, GithubException
import pandas as pd
import config

## Estrategia

Como el objetivo de nuestra tarea es la generación de frases *mañaneras* del presidente Andres Manuel Lopez Obrador, solo utilizaremos los datos relacionados al presidente. Dichos datos los podemos encontrar con la siguiente ruta.

`repo-root/año/mes/dia/csv_por_participante/AMLO.csv`

Para visualizar como vienen los datos, primero obtenemos un DataFrame.


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/NOSTRODATA/conferencias_matutinas_amlo/master/2019/11-2019/noviembre%201%2C%202019/csv_por_participante/PRESIDENTE%20ANDRES%20MANUEL%20LOPEZ%20OBRADOR.csv', encoding='utf-8')

Visualizamos la información

In [3]:
df.head(5)

,Participante,Texto,Sentimiento,Palabras,Dia,Mes,Anio
0,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Buenos días. Ánimo.,0.328237,3,1,11,2019
1,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Vamos a continuar con la sesión de ayer acerca...,0.328237,29,1,11,2019
2,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Y también vamos a dar a conocer a todos que se...,0.328237,77,1,11,2019
3,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,"Ricardo Rodríguez Vargas, que es el director d...",0.328237,35,1,11,2019
4,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Muy bien.,0.328237,2,1,11,2019


De aqui observamos que para la creación de nuestro corpus solamente utilizaremos la columna `Texto` del DataFrame.

Creamos una función que permita almacenar el contendio de una columna de un DataFrame, y que concatene al archivo especificado.


In [4]:
def append_to_file(df, column, file_path):
    if file_path == '':
        raise ValueError("Invalid file path")
    values = df[column].values.tolist()
    with open(file_path, mode='a', encoding='utf-8') as f:
        for line in values:
            f.write(line + '\n')
        f.write('\n')

Esta función la utilizaremos para que el archivo `.txt` quede de forma:

 * archivo1.csv
 * archivo2.csv
 * ...
 * archivo_n.csv

Ahora realizamos un script para obtener los datos utilizando `PyGitHub`

Creamos la conexión con GitHub mediante un token personal a nuestra cuenta de GitHub.

In [5]:
g = Github(config.GH_TOKEN)
repo = g.get_repo('NOSTRODATA/conferencias_matutinas_amlo') 

In [6]:
expectedEntries = 0
entriesSaved = 0

In [7]:
years = [str(i) for i in range(2018, 2024)]

In [8]:
for year in years:
    months_available = repo.get_contents(year)
    for month in months_available:
        days_available = repo.get_contents(year + "/" + month.name)
        expectedEntries += len(days_available)
        for day in days_available:
            path = year + "/" + month.name + '/' + day.name + '/' + 'csv_por_participante'
            try:
                print(f"Trying {path + '/' + 'PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR.csv'}")
                file = repo.get_contents(path + '/' + 'PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR.csv')
            except GithubException as e:
                files = [file.name for file in repo.get_contents(path) if 'PRESIDENTE' in file.name]
                if not files:
                    print(f'File not found...')
                    continue
                print(f'File not found... Trying {path + "/" + files[0]}')
                file = repo.get_contents(path + "/" + files[0])
            
            raw_url = file.download_url
            try:
                df = pd.read_csv(raw_url, encoding='utf-8')
            except:
                pass
            append_to_file(df, 'Texto', '../data/corpus.txt')
            entriesSaved += 1

Trying 2018/12-2018/diciembre 10, 2018/csv_por_participante/PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR.csv
Trying 2018/12-2018/diciembre 26, 2018/csv_por_participante/PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR.csv
Trying 2018/12-2018/diciembre 27, 2018/csv_por_participante/PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR.csv
Trying 2018/12-2018/diciembre 28, 2018/csv_por_participante/PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR.csv
Trying 2018/12-2018/diciembre 4, 2018/csv_por_participante/PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR.csv
File not found... Trying 2018/12-2018/diciembre 4, 2018/csv_por_participante/-PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR.csv
Trying 2018/12-2018/diciembre 6, 2018/csv_por_participante/PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR.csv
File not found... Trying 2018/12-2018/diciembre 6, 2018/csv_por_participante/-PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR.csv
Trying 2018/12-2018/diciembre 7, 2018/csv_por_participante/PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR.csv
File not found... Trying 2018/12-2018/di

In [9]:
print(f"Esperaba {expectedEntries} pero guarde {entriesSaved}")

Esperaba 1007 pero guarde 1005
